# Zadanie 4

Rozpatrzymy przykład sieci, której zadaniem będzie klasyfikacja pary liczb do dwóch grup. Albo grupy, której wynik operacji XOR da 0 lub grupy, której wynik operacji XOR da 1. XOR polega ona na przyjęciu: dwóch liczb (zazwyczaj są to liczby naturalne 0 lub 1) oraz tego, że oczekiwany wynik jest równy zeru, gdy dwie liczby są takie same. Dla par (0,0) i (1,1) otrzymujemy na wyjściu 0, natomiast dla par różniących się (0,1) lub (1,0) otrzymujemy na wyjściu 1. Problem pojawia się w momencie, gdy założymy, że nasze liczby nie są liczbami całkowitymi, to znaczy jak mamy interpretować parę liczb (0.1,1.3)? Wyobraźmy sobie, że musimy dokonać operacji XOR dwóch wejść, które podają napięcie z zakresu od 0V do 1.3V. To napięcie prawie zawsze nie będzie idealnie równe liczbie całkowitej. Dla takiego układu musimy stworzyć model, który będzie potrafił sklasyfikować (stąd nazwa sieć klasyfikacyjna lub klasyfikator) daną parę liczb do dwóch grup – albo do grupy par liczb, dających w wyniku operacji XOR 0 lub do grupy par liczb, które w wyniku operacji XOR dadzą 1. Będziemy realizować cel wygenerowania losowych danych, na których dokonamy późniejszego treningu (uczenia) naszej sieci:

In [ ]:
import torch
import torch.utils.data as data

class XORDataCreator(data.Dataset):
   """  XORDataCreator jest klasą, która generuje losowe dane treningowe dla głębokiej sieci klasyfikacyjnej.
    Klasyfikator, który korzysta z generowanych danych powinien otrzymać parę liczb zmiennoprzecinkowych obarczonych szumem oraz oznaczenie pary liczb jakiego wyniku alternatywy rozłącznej (operacja XOR) spodziewamy się po tej parze. Na tej podstawie sieć ma nauczyć się rozpoznawać wyniki z możliwie jak najwyższą poprawnością nieznanej dotąd pary liczb zmiennoprzecinkowych."""
    # Definicja konstruktora (metoda inicjalizacyjna klasy)
    # data_size: ilość danych losowych, która ma zostać wygenerowana przez obiekt tej klasy
    # noise_std_deviation: odchylenie standardowe szumu Gaussowskiego (zaszumienie danych ma na celu przygotować trening sieci na mniej standardowe dane)
    def __init__(self, data_size, noise_std_deviation) -> None:
        super().__init__()
        self.data_size = data_size
        self.noise_std_deviation = noise_std_deviation
        self.generate_random_xor_data()
    def generate_random_xor_data(self):
        # Generacja losowych par liczb (x,y) z przedziału <0,2>
        xor_data = torch.randint(low=0, high=2, size=(self.data_size, 2), dtype=torch.float32)
        # Generacja szumu dla danych (losowy tensor o tych samych wymiarach co nasze dane wygenerowane linię wyżej z zakresu <0,1> pomnożony przez odchylenie standardowe)
        xor_data_noise = ???
        # Labelling danych oznacza nadanie każdemu elementowi z tablicy danych oczekiwanego wyniku
        # W tym przypadku aby określić oczekiwany wynik dokonujemy operacji XOR (operator ^).
        labels = ???
        # Zapisujemy wygenerowane dane do pól klasy prezentujących zbiór danych treningowych oraz nadanym im oczekiwanych wyników (labels)
        xor_data_with_noise = ???
        labels_converted_to_numbers = labels # Lista 'labels' zawiera jednoelementowe tensory - poprzez metodę item() wyłuskujemy wartość
        self.data_inputs = xor_data_with_noise
        self.data_labels = [label.to(torch.long) for label in labels]
    def __len__(self):
        return ???
    def __getitem__(self, index):
        return ???
# Użycie wyżej zdefiniowanej klasy do generacji 300 elementowego tensora danych
xor_dataset = ???

Po uruchomieniu naszego program zmienna xor_dataset powinna zawierać tensor (a dokładniej parę określaną jako tuple) dwóch elementów – jeden z nich to kolejny tensor, który zawiera zbiór 300 elementów par losowo wygenerowanych liczb, a drugi element to oczekiwany wynik operacji XOR pary liczb z pierwszego tensora. Oczekiwany wynik jest podawany w sieci, aby ta mogła na bieżąco „dostrajać” wagi w tym przypadku dwóch wejść (na wejściu mamy parę liczb) tak, aby wyjście z jak największym prawdopodobieństwem pokrywało się z tym, czego oczekujemy na podstawie wygenerowanych danych.

ak widać, pary liczb, które są różne (zaokrąglają się do różnych liczb całkowitych) i wedle operacji XOR dają wynik 1 (pomarańczowe punkty), znajdują się w lewej górnej oraz prawej dolnej części wykresu, przeciwnie do par, które zaokrąglają się do tych samych wartości. Teraz zajmiemy się próbą zdefiniowania architektury oraz działania naszego modelu głębokiej sieci neuronowej tak, aby ta mogła dla dowolnego punktu danego tej sieci (nawet takiego, którego nie było w danych, na których sieć była uczona) określić, do jakiej grupy należy.

Oczywiście, ten problem jest stosunkowo prosty celem zaprezentowania zwięzłego przykładu, ale każdy problem, który da się opisać liczbowo, może później zostać poddany próbie nauczenia sieci tego, w jaki sposób te dane są powiązane z informacją końcową, która interesuje użytkownika (na przykład wyniki badań krwi z daną jednostką chorobową).

Skupimy się teraz na zbudowaniu modelu sieci neuronowej oraz zdefiniowaniu procesu treningu (uczenia) sieci. Pierwszym szczegółem jest użycie modułu torch.nn – skrót „nn” pochodzi od pojęcia neural network. Jest to moduł, który pozwala nam w łatwy sposób zdefiniować strukturę naszego modelu sieci neuronowej. W naszym przypadku, ponieważ jest to dość prosty problem, jest to struktura składająca się z:
    • Warstwy wejściowej (wejściowa para liczb),
    • Warstwy zwanej ukrytą (wszystkie warstwy głębokich sieci neuronowych poza warstwą wejściową i wyjściową nazywane są zwyczajowo ukrytymi),
    • Warstwą wyjściową, która prezentuje już interesujący nas wynik.
Poniżej zdefiniowana klasa ClassifierModel definiuje model naszej głębokiej sieci klasyfikatora. 
W konstruktorze za argumenty wejściowe przyjmujemy ilość neuronów poszczególnych warstw (ilość komórek przyjmujących dane oraz propagujących je dalej w przetworzonej formie). Skupmy się na wyjaśnieniu zdefiniowanych relacji między warstwami – tutaj widać konkretne odwołanie do modułu nn zawierający wiele modeli klas, które pozwalają nam w prosty i przejrzysty sposób zaprojektować naszą sieć.

In [ ]:
class ClassifierModel(nn.Module):
    def __init__(self, inputs_number, hidden_layer_neurons_number, outputs_number) -> None:
        super().__init__()
        self.first_layer_to_hidden_transformation = nn.Linear(inputs_number, hidden_layer_neurons_number) 
        self.activation_function = ???
        self.hidden_layer_to_output_transformation = ???
    def forward(self, input):
        x = self.first_layer_to_hidden_transformation(input)
        x = self.activation_function(x)
        x = self.hidden_layer_to_output_transformation(x)
        return x

Idąc kolejno po każdej linii naszej klasy, napotykamy na definicję pierwszej transformacji (na czerwono). Definiujemy tę transformację jako liniową (nn.Linear), czyli taką, która jest standardową definicją wiążącą wejścia, które są ważone (nadawane są im wagi, czyli liczby, przez które są mnożone wejścia, a ustalane są przez sieć w procesie uczenia tak, aby jak najlepiej odzwierciedlać wyuczonym modelem opisaną w danych treningowych przez nas sytuację), a zaprezentowane mogą zostać za pomocą dość prostego wzoru funkcji liniowej. Funkcja liniowa poprzez odpowiedni dobór wag w wektorze W stara się zminimalizować błąd (pomyłkę) modelu sieci na podstawie dostarczonych danych treningowych. Opisywany problem jest dość prosty i da się opisać analitycznie w sposób liniowy, natomiast niestety większość musi zostać opisana za pomocą nieliniowych funkcji. Następną ważną składową naszej sieci jest funkcja aktywacji, która w naszym przypadku jest funkcją równą tangensowi hiperbolicznemu (nn.Tanh()). Wartości naszej funkcji aktywacji wchodzą w zakres pomiędzy -1 a 1. Jest to świetna funkcja, która wprowadza poprzez swoją nieliniowość możliwość rozwiązania nieliniowych problemów. W dodatku jest wycentrowaną funkcją, ponieważ dla argumentu zero również przyjmuje wartość równą zeru — przecina osie dokładnie w punkcie (0,0). Z tego powodu, podczas gdy nasza sieć się uczy i próbuje poznać, jak wrażliwa jest funkcja błędu (pomyłki) na wspomniane wcześniej wagi każdego z wejść (licząc gradient tej funkcji), korzysta z przywileju, iż gradient tangensu hiperbolicznego jest symetryczny względem osi zerowej (y = 0).

To wielka przewaga tej funkcji nad wieloma pozostałymi nieliniowymi funkcjami aktywacji. Jednakże różne klasy problemów do rozwiązania mają swoje empiryczne rekomendacje odnośnie zastosowania konkretnej funkcji aktywacji. W internecie możemy spotkać mnóstwo dobrej jakości poradników w tym zakresie i warto z nich korzystać, ponieważ zazwyczaj są one poparte praktyką.

Ostatnią transformacją wiążącą warstwę ukrytą i warstwę wyjściową jest również relacja liniowa. Funkcja forward zdefiniowana w tej samej klasie ma za zadanie przyjąć wektor wejściowy i „przepuścić” go przez kolejne warstwy, po czym zwrócić tensor wyjściowy. Taki tensor jest dla nas po prostu informacją o tym, jaki wynik operacji XOR otrzymamy. Wynikiem będzie liczba zmiennoprzecinkowa pomiędzy 0 a 1, gdyż wynik operacji XOR to właśnie w idealnym przypadku 0 lub 1 – tutaj sieć o tym nie wie, dlatego z pewnym prawdopodobieństwem przybliża się ku zeru lub jedynce, stąd zmiennoprzecinkowość.


W skrócie, cały proces przygotowania oraz treningu naszej sieci będzie przebiegał w następujących krokach:

Inicjalizacja klasy odpowiedzialnej za dostarczenie danych treningowych.
Pobranie częściowej paczki danych z całościowej kolekcji danych treningowych sieci.
Obliczenie przez obecny model sieci wyjścia dla zaprezentowanych danych wejściowych.
Obliczenie błędu, czyli różnicy pomiędzy wynikiem oczekiwanym (podanym na przykład przez nas w klasie odpowiedzialnej za dostarczenie danych – będzie to na przykład dla pary liczb (0,1) oczekiwany wynik 1, ponieważ operacja XOR dla tej pary daje właśnie taki wynik) a tym, jaki wynik otrzymaliśmy w punkcie 3.
Dokonanie propagacji wstecznej błędu, czyli sprawdzenie na podstawie funkcji gradientu (czyli funkcji prezentującej wrażliwość funkcji błędu na wagi nadane konkretnym wejściom), w którym punkcie jesteśmy (czy zwiększając na przykład wagę danego wejścia zwiększymy, czy zmniejszymy błąd – będziemy zawsze chcieli go zmniejszyć).
Aktualizacja wag tak, aby zmniejszyć błąd sieci.
Gdy błąd jest na tyle mały, że uznamy go za satysfakcjonujący, zapisujemy model sieci.
Punkty te będziemy powtarzać cyklicznie aż osiągniemy zadowalającą dokładność przewidywania wyniku przez model sieci.
Przejdźmy zatem do kluczowej części kodu, czyli treningu naszej sieci. W tym celu, posługując się powyższą instrukcją, przeanalizujemy kod, którego zadaniem będzie w ogólności trening naszego modelu sieci.

In [ ]:
import torch
import torch.nn as nn
from classifier_model import ClassifierModel
from data_loader import XORDataCreator

class Net():

    def __init__(self) -> None:
        self.loss_calculation_model = None
        self.optimizer = None
        self.data_loader = XORDataCreator(data_size=100, noise_std_deviation=0.1)

    def train(self, model: ClassifierModel, epochs_num: int=50):
        # Sprawdźmy czy możemy wykorzystać GPU poprzez pakiet CUDA celem przyspieszenia obliczeń
        gpu_available = torch.cuda.is_available()
        # W przypadku możliwości ustawmy 'device' na GPU
        device = torch.device('cuda') if gpu_available else torch.device('cpu')

        # Inicjalizacja
        self.loss_calculation_model = nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

        # Załadujmy nasz model do GPU jeśli jest dostępne
        model.to(device)
        # Ustawmy model w tryb treningowy (funkcjonalność odziedziczona po klasie nn.Module)
        model.train()

        # Pętla treningowa (pobierz paczkę danych treningowych)
        for current_epoch in range(epochs_num):
            epoch_loss = 0.0
            for data_input, data_label in self.data_loader:
                # Załadujmy nasze dane do GPU jeśli jest w użyciu
                data_input = data_input.to(device)
                data_label = data_label.to(device)

                # Przeprocedujmy dane wejściowe przez nasz model
                predicted_output = ???
                predicted_output = ???

                # Obliczmy wartość funkcji straty (jak bardzo nasz model się pomylił)
                loss = self.loss_calculation_model(predicted_output, data_label.float())

                # Wyzerujmy wartość gradientu na wszelki wypadek
                ???

                # Dokonajmy propagacji wstecznej błędu
                ???

                # Zaktualizujmy wagi sieci na podstawie obliczonego gradientu
                ???

                # Obliczenie średniej pomyłki obecnej iteracji treningowej
                epoch_loss += ???

            # Logowanie istotnych danych odnośnie zakończonej iteracji
            print('epoch: ', ???)
            print('average epoch loss: ', ???)

    def set_optimizer(self, optimizer) -> None:
        self.optimizer = ???

    def set_loss_calc_model(self, loss_calc_model) -> None:
        self.loss_calculation_model = ???

    def get_training_data(self) -> XORDataCreator:
        return ???

Zazwyczaj trening skomplikowanych modeli sieci neuronowych jest czasochłonny – zatem pozostaje pytanie, co można zrobić, aby nie utracić danych treningowych w losowych przypadkach lub aby móc podzielić ten czasochłonny proces na kilka mniejszych? Oczywiście można zapisać parametry tymczasowego modelu do pliku, a następnie wczytać ostatnie zapisane podczas ponownej sesji treningowej. Aby wczytać oraz zapisać stan bieżący modelu, należy użyć wbudowanych metod pakietu torch.

In [ ]:
from os.path import exists
import torch
from net import Net
from classifier_model import ClassifierModel
 
__clasifier_model_state_file_name__ = "classifier_model.pt"
 
classifier_model = ClassifierModel(???)
 
# Wczytaj poprzednio zapisany model, jeśli istnieje
if exists(__clasifier_model_state_file_name__):
    model_state = torch.load(???)
    classifier_model.load_state_dict((???)


net_model = ???
net_model.train_and_log(model=???)
 
# Zapisz dotychczasowy model
model_state = classifier_model.state_dict()
torch.save(???)
print("Trained model state dictionary saved to ", ???)